In [1]:
import numpy as np
import argparse
import cv2 as cv
from skimage.exposure import rescale_intensity
from matplotlib import pyplot as plt

In [2]:
def convolve_divRGB(image, kernel,pad,stride):
    b,g,r = cv.split(img)
    b = convolve_main(b, kernel,pad,stride)
    g = convolve_main(g, kernel,pad,stride)
    r = convolve_main(r, kernel,pad,stride)
    output = cv.merge((b,g,r))
    return output

In [3]:
def convolve_main(image, kernel,pad,stride):

    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    
    if pad==0:
        arange_size = pad
        ch_position = 2
    elif pad==1:
        arange_size = pad
        ch_position = 0
    else:
        arange_size = 2*pad
        ch_position = 0
    
    image = cv.copyMakeBorder(image, pad, pad, pad, pad, cv.BORDER_CONSTANT)
    oH = int((iH + 2*pad-kH)/stride + 1)
    oW = int((iW + 2*pad-kW)/stride + 1)
    output = np.zeros((oH, oW),dtype="float32")
    for y in np.arange(pad+ch_position, iH + arange_size-ch_position,stride):
        for x in np.arange(pad+ch_position, iW + arange_size-ch_position,stride): 
            roi = image[y-pad: y-pad+3, x-pad: x-pad+3]
            k = (roi*kernel).sum()
            temp_y = int((y-pad)/stride)
            temp_x = int((x-pad)/stride)
            output[temp_y, temp_x]=k
    output = rescale_intensity(output, in_range=(0,255))
    output = ((output*255).astype("uint8"))
           # print output
    return output

In [5]:
img = cv.imread('lena.jpg')
# gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
kernel = np.array([[-1,-1,-1],
                   [-1,8,-1],
                   [-1,-1,-1]])

dst = convolve_divRGB(img, kernel,10,1)
dst2 = cv.filter2D(img, -1, kernel)
cv.imshow('ori',img)
cv.imshow('my_convol',dst)
cv.imshow('filter2D_convol',dst2)
cv.waitKey(0)
cv.destroyAllWindows()